In [1]:
import requests
import torch
import torch.nn as nn


In [2]:
def get_heroes():
    url = "https://api.opendota.com/api/heroes"
    response = requests.get(url)
    if response.status_code == 200:
        heroes = response.json()
        hero_dict = {hero['localized_name']: hero['id'] for hero in heroes}
        return hero_dict
    else:
        return None
class Dota2Model(nn.Module):
    def __init__(self, num_heroes, embedding_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_heroes, embedding_dim=embedding_dim)
        
        input_dim = (embedding_dim * 2) + 1 

        self.layers = nn.Sequential(
            nn.Linear(in_features=input_dim, out_features=hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.ReLU(),
            nn.Linear(hidden_dim // 2, hidden_dim // 4),
            nn.ReLU(),
            nn.Linear(hidden_dim // 4, out_features=hidden_dim // 8),
            nn.ReLU(),
            nn.Linear(hidden_dim // 8, out_features=hidden_dim // 8),
            nn.ReLU(),
            nn.Linear(hidden_dim // 8, out_features=hidden_dim // 8),
            nn.Linear(hidden_dim // 8, out_features=hidden_dim // 8),
            nn.Linear(hidden_dim // 8, out_features=1)
        )

    def forward(self, match):
        avg_rank_tier = match[:, 0]
        radiant_teams = match[:, 1:6]
        dire_teams = match[:, 6:]
        
        radiant_embeds = self.embedding(radiant_teams)
        dire_embeds = self.embedding(dire_teams)
        
        radiant_embeds = radiant_embeds.mean(dim=1)
        dire_embeds = dire_embeds.mean(dim=1)
        
        combined_teams = torch.cat([avg_rank_tier.unsqueeze(1), radiant_embeds, dire_embeds], dim=1)
        
        return self.layers(combined_teams)

heroes = get_heroes()
ranks = {"Herald": 0, "Guardian": 5, "Crusader": 10, "Archon": 15, "Legend": 20, "Ancient": 25, "Divine": 30, "Immortal": 35}
dota_2_model = Dota2Model(max(heroes.values())+1, 256, 256)
dota_2_model.load_state_dict(torch.load("model.pth"))

C:\Users\nobod\AppData\Local\Temp\ipykernel_10420\1195991031.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  dota_2_model.load_state_dict(torch.load("model.pth"))


<All keys matched successfully>

In [3]:
print("Choose avg rank")
for rank in ranks:
    print(rank)
rank = ranks[input("Avg rank is: ")]
print("How many stars you have?")
rank += int(input())
game = [rank]
print(heroes.keys())
print("Choose radiant team")
for i in range(5):
    game.append(heroes[input()])
print("Choose dire team")
for i in range(5):
    game.append(heroes[input()])
game = torch.Tensor(game)

Choose avg rank
Herald
Guardian
Crusader
Archon
Legend
Ancient
Divine
Immortal


Avg rank is:  Immortal


How many stars you have?


 1


dict_keys(['Anti-Mage', 'Axe', 'Bane', 'Bloodseeker', 'Crystal Maiden', 'Drow Ranger', 'Earthshaker', 'Juggernaut', 'Mirana', 'Morphling', 'Shadow Fiend', 'Phantom Lancer', 'Puck', 'Pudge', 'Razor', 'Sand King', 'Storm Spirit', 'Sven', 'Tiny', 'Vengeful Spirit', 'Windranger', 'Zeus', 'Kunkka', 'Lina', 'Lion', 'Shadow Shaman', 'Slardar', 'Tidehunter', 'Witch Doctor', 'Lich', 'Riki', 'Enigma', 'Tinker', 'Sniper', 'Necrophos', 'Warlock', 'Beastmaster', 'Queen of Pain', 'Venomancer', 'Faceless Void', 'Wraith King', 'Death Prophet', 'Phantom Assassin', 'Pugna', 'Templar Assassin', 'Viper', 'Luna', 'Dragon Knight', 'Dazzle', 'Clockwerk', 'Leshrac', "Nature's Prophet", 'Lifestealer', 'Dark Seer', 'Clinkz', 'Omniknight', 'Enchantress', 'Huskar', 'Night Stalker', 'Broodmother', 'Bounty Hunter', 'Weaver', 'Jakiro', 'Batrider', 'Chen', 'Spectre', 'Ancient Apparition', 'Doom', 'Ursa', 'Spirit Breaker', 'Gyrocopter', 'Alchemist', 'Invoker', 'Silencer', 'Outworld Destroyer', 'Lycan', 'Brewmaster', '

 Ursa
 Sand King
 Nature's Prophet
 Monkey King
 Weaver


Choose dire team


 Void Spirit
 Queen of Pain
 Timbersaw
 Ringmaster
 Tinker


Heroes from 7917084099

In [4]:
dota_2_model.eval()
with torch.inference_mode():
    y_logit = dota_2_model(game.unsqueeze(0).long())
    y_pred = torch.sigmoid(y_logit)
print(f"Radiant chance to win {y_pred.squeeze(1).item()*100}%")

Radiant chance to win 28.933444619178772%
